In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from pathlib import Path
from detection_helper import create_DF
from matplotlib import pyplot as plt
import numpy as np

In [2]:
train_files = list(Path("Dataset/train/").glob("**/*txt"))
val_files = list(Path("Dataset/validation/").glob("**/*txt"))

In [3]:
df_train = create_DF(train_files,Path("Dataset/train/Apple_Grape_Cantaloupe_Watermelon_Pomegranate_Pineapple_Peach_Mango_Banana_Lemon_Tomato_Strawberry_Pear/"))

In [4]:
df_val = create_DF(val_files,Path("Dataset/validation/Apple_Grape_Cantaloupe_Watermelon_Pomegranate_Pineapple_Peach_Mango_Banana_Lemon_Tomato_Strawberry_Pear/"))

In [5]:
df_train.head()

,label,left,top,right,bottom,height,width,image,image_name,center_x,center_y,delta_x,delta_y
0,Apple,236.16,0.000000,625.92,342.140654,1024,683,Dataset/train/Apple_Grape_Cantaloupe_Watermelo...,837bf3be7055dc46,431.04,171.070327,389.76,342.140654
1,Apple,318.72,44.209224,466.56,246.674329,1024,683,Dataset/train/Apple_Grape_Cantaloupe_Watermelo...,837bf3be7055dc46,392.64,145.441777,147.84,202.465105
2,Grape,285.44,390.194485,469.12,565.749390,1024,683,Dataset/train/Apple_Grape_Cantaloupe_Watermelo...,116b98c8f1a2538c,377.28,477.971938,183.68,175.554905
3,Strawberry,321.28,484.231821,493.44,654.609170,1024,841,Dataset/train/Apple_Grape_Cantaloupe_Watermelo...,b1e2809b606fb594,407.36,569.420496,172.16,170.377349
4,Strawberry,111.36,332.428798,187.52,450.924857,1024,841,Dataset/train/Apple_Grape_Cantaloupe_Watermelo...,b1e2809b606fb594,149.44,391.676828,76.16,118.496059


In [6]:
df_val.head()

,label,left,top,right,bottom,height,width,image,image_name,center_x,center_y,delta_x,delta_y
0,Tomato,72.387584,119.736162,793.486336,619.030503,1024,819,Dataset/validation/Apple_Grape_Cantaloupe_Wate...,604835415b761f30,432.936960,369.383333,721.098752,499.294341
1,Lemon,89.567232,152.054784,441.594880,461.195520,1024,768,Dataset/validation/Apple_Grape_Cantaloupe_Wate...,b39532c4a3bd5d83,265.581056,306.625152,352.027648,309.140736
2,Strawberry,0.000000,0.053248,1023.358976,1024.000000,1024,1024,Dataset/validation/Apple_Grape_Cantaloupe_Wate...,e0be14f0681f4b39,511.679488,512.026624,1023.358976,1023.946752
3,Lemon,236.441600,90.215200,905.010176,573.973625,1024,575,Dataset/validation/Apple_Grape_Cantaloupe_Wate...,730020d358ea3d82,570.725888,332.094412,668.568576,483.758425
4,Lemon,320.348160,30.697525,724.350976,119.152075,1024,575,Dataset/validation/Apple_Grape_Cantaloupe_Wate...,730020d358ea3d82,522.349568,74.924800,404.002816,88.454550


In [7]:
classes = df_train["label"].unique()
classes

array(['Apple', 'Grape', 'Strawberry', 'Banana', 'Lemon', 'Pomegranate',
       'Mango', 'Peach', 'Tomato', 'Pear', 'Watermelon', 'Pineapple',
       'Cantaloupe'], dtype=object)

In [11]:
classes_idx = {clase:idx for idx,clase in enumerate(classes)}

In [13]:
cant_classes = len(classes)
cant_classes

13

In [14]:
import pickle
with open("data.pkl","wb") as f:
    pickle.dump((df_train, df_val, train_files, val_files, classes_idx, cant_classes, classes),f)